In [1]:
from nanotron.dataloader import (
    clm_process,
    dummy_infinite_data_generator,
    get_datasets,
    get_train_dataloader,
)

from transformers import AutoTokenizer

In [5]:
dataset = "cmarkea/doc-vqa"
tokenizer_path = "robot-test/dummy-tokenizer-wordlevel"

In [6]:
import os

# os.environ["HF_ENDPOINT"] = "http://localhost:5564"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [7]:
raw_dataset = get_datasets(
    hf_dataset_or_datasets=dataset,
    hf_dataset_config_name=None,
    splits="train",
)["train"]

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

README.md:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

train-00000-of-00013.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

train-00001-of-00013.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

train-00002-of-00013.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00003-of-00013.parquet:   0%|          | 0.00/139M [00:00<?, ?B/s]

...

test-00001-of-00004.parquet:   0%|          | 0.00/111M [00:00<?, ?B/s]

test-00002-of-00004.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

test-00003-of-00004.parquet:   0%|          | 0.00/95.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9688 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2421 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoProcessor, AutoModelForVision2Seq

In [10]:
processor = AutoProcessor.from_pretrained("HuggingFaceM4/Idefics3-8B-Llama3")

In [26]:
ex = raw_dataset[0]
messages = []
for i, x in enumerate(ex["qa"]["en"]):
    user_message = {
        "role": "user",
        "content": [
            {"type": "text", "text": x["question"]},
        ]
    }

    if i == 0:
        user_message["content"].append(
            {"type": "image"},
        )

    messages.append(user_message)
    assistant_message = {
        "role": "assistant",
        "content": [
            {"type": "text", "text": x["answer"]},
        ]
    }

    messages.append(assistant_message)

In [27]:
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[ex["image"]], return_tensors="pt")
inputs = {k: v.to("cpu") for k, v in inputs.items()}

In [32]:
inputs = processor(text=prompt, images=[ex["image"]], return_tensors="pt", max_length=128, padding="longest", truncation=True)
inputs["input_ids"].shape
inputs["attention_mask"].shape


torch.Size([1, 128])

In [12]:
pretraining_data = [
    {
        "image": None,
        "text": "A view of the Statue of Liberty in New York."
    },
    {
        "image": None,
        "text": "The skyline of Chicago during sunset."
    },
    # ... more image-text pairs
]


text = processor.apply_chat_template(pretraining_data, add_generation_prompt=True)
# inputs = processor(images=images, text=text, return_tensors="pt").to(DEVICE)

UndefinedError: 'dict object' has no attribute 'role'

In [13]:
new_c = ["2313212"] * len(raw_dataset)

new_dataset = raw_dataset.add_column("new_c", new_c)

In [14]:
train_dataset = clm_process(
    raw_dataset=new_dataset,
    tokenizer=tokenizer,
    text_column_name="new_c",
    dataset_processing_num_proc_per_process=16,
    dataset_overwrite_cache=False,
    sequence_length=256,
)

Grouping texts in chunks of 257 (num_proc=16):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [16]:
len(train_dataset)

320